## <b>RAG with Azure OpenAI<b>

### Prerequisite package installation 

In [ ]:
# ! pip install llama-index-embeddings-azure-openai
# ! pip install llama-index-llms-azure-openai

In [ ]:
# ! pip install llama-index

###  Loading Dotenv package to load environment variables from .env file

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Loading Necessary packages

In [2]:
import os
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,StorageContext,load_index_from_storage,get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Here, we setup the embedding model (for retrieval) and llm (for text generation).
Note that you need not only model names (e.g. "text-embedding-ada-002"), but also model deployment names (the one you chose when deploying the model in Azure.
You must pass the deployment name as a parameter when you initialize `AzureOpenAI` and `OpenAIEmbedding`.

### Initializing the AzureOpenAI and OpenAIEmbedding models 

In [3]:
api_key = os.getenv('AZURE_API_KEY')
azure_endpoint = os.getenv('AZURE_ENDPOINT')
api_version = "2023-07-01-preview"

llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt-4",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    temperature=0.4,
    max_tokens=500
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version
)

In [4]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [13]:
response=llm.complete("Wheels on the bus go...")

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/gpt-4/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/gpt-4/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [14]:
response

CompletionResponse(text='The wheels on the bus go round and round, round and round, round and round.', additional_kwargs={}, raw={'id': 'chatcmpl-90bBKz1XGvtPFmq4M9TE5IN3o2G9x', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The wheels on the bus go round and round, round and round, round and round.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], 'created': 1709929954, 'model': 'gpt-4', 'object': 'chat.completion', 'system_fingerprint': 'fp_8abb16fa4e', 'usage': CompletionUsage(completion_tokens=18, prompt_tokens=14, total_tokens=32), 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'se

### Data Ingestion from source directory

In [16]:
documents = SimpleDirectoryReader("./data").load_data()

In [17]:
len(documents)

42

### Splitting documents into chunks

In [18]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=100,chunk_overlap=0)

nodes = parser.get_nodes_from_documents(documents)

In [19]:
len(nodes)

230

### Importing FAISS Vector Store

In [20]:
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [21]:
import faiss

# dimensions of text-ada-embedding-002
d = 1536

# Creating an index with flat indices and metric as 'Cosine similarity' (IP: Inner Product)
faiss_index= faiss.IndexFlatIP(d)

INFO:faiss.loader:Loading faiss with AVX2 support.
Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
Successfully loaded faiss with AVX2 support.


### Indexing the documents and creating the vector store

In [172]:
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context, 
    embed_model=embed_model, 
    show_progress=True,
)

Generating embeddings:   0%|          | 0/230 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings:   4%|▍         | 10/230 [00:01<00:42,  5.21it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings:   9%|▊         | 20/230 [00:02<00:23,  9.11it/s]

...

Generating embeddings:  91%|█████████▏| 210/230 [00:13<00:01, 19.15it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings:  96%|█████████▌| 220/230 [00:13<00:00, 19.18it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings: 100%|██████████| 230/230 [00:14<00:00, 16.20it/s]


In [134]:
print(storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x0000022A6B604350>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x0000022A6B604C90>, vector_stores={'default': FaissVectorStore(stores_text=False, is_embedding_query=True), 'image': <llama_index.core.vector_stores.simple.SimpleVectorStore object at 0x0000022A6B604650>}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x0000022A6B605490>)


### Storing the index on the disk for future use

In [173]:
index.storage_context.persist()

### Loading the index from the disk

In [22]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

INFO:root:Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [23]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)
response_synthesizer = get_response_synthesizer()

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

In [24]:
from llama_index.core.evaluation import FaithfulnessEvaluator
evaluator = FaithfulnessEvaluator(llm=llm)

In [182]:
response = query_engine.query("who is the president of india")

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mathurai.openai

In [183]:
display(Markdown(f"<b>{response}</b>"))

<b>I'm sorry, I cannot answer that question as it is not related to the provided context information about life insurance and personal information.</b>

In [25]:
response = query_engine.query("what are insurance policies?")

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/gpt-4/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/gpt-4/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [26]:
display(Markdown(f"<b>{response}</b>"))

<b>Insurance policies are legal agreements between an individual and an insurance company, guaranteeing the payment of a specified amount upon certain events, such as the policyholder's death. These policies come in various types, including permanent and term life insurance. Permanent life insurance offers coverage for the policyholder's entire lifetime, with variations such as whole life, universal life, and variable life, provided the policy is kept in force. Term life insurance, on the other hand, provides coverage for a specific period, paying out a death benefit only if the policyholder dies within that term. The premiums and benefits of these policies can vary, with some policies offering guarantees on premiums and death benefits for certain periods, adjusting based on investment conditions afterward. Additionally, life insurance benefits are typically tax-free to the beneficiary.</b>